In [1]:
#Description: This is a sentiment analysis :Detecting the emotion of text


In [7]:
#Import the libraries
!pip install nlp
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nlp
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
#Importing and Preparing Data
dataset = nlp.load_dataset('emotion')
train = dataset['train']
val = dataset['validation']
test = dataset['test']

Downloading:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.01k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/204k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/84e07cd366f4451464584cdbd4958f512bcaddb1e921341e07298ce8a9ce42f4. Subsequent calls will reuse this data.


In [10]:
#separates it into the training, validation, and testing sets
def get_tweet(data):
 tweets = [x['text'] for x in data]
 labels = [x['label'] for x in data]
 return tweets, labels
tweets, labels = get_tweet(train) 

In [14]:
tweets[0],labels[0]

('i didnt feel humiliated', 'sadness')

In [11]:
#Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(tweets)

In [12]:
tweets[0]

'i didnt feel humiliated'

In [13]:
tokenizer.texts_to_sequences([tweets[0]])

[[2, 139, 3, 679]]

In [16]:
#Making all Sequences Same Shape
maxlen=50
def get_sequences(tokenizer, tweets):
 sequences = tokenizer.texts_to_sequences(tweets)
 padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
 return padded 

In [18]:
padded_train_seq=get_sequences(tokenizer,tweets)
padded_train_seq[0]

array([  2, 139,   3, 679,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [19]:
#Preparing Data for Model
classes = set(labels)
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])
train_labels = names_to_ids(labels)


In [20]:
print(classes)

{'surprise', 'sadness', 'anger', 'fear', 'joy', 'love'}


In [21]:
index_to_class


{0: 'surprise', 1: 'sadness', 2: 'anger', 3: 'fear', 4: 'joy', 5: 'love'}

In [22]:
train_labels[0]

1

In [24]:
#Creating Model
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,16,input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [30]:
#Training Model
val_tweets, val_labels = get_tweet(val)
val_seq = get_sequences(tokenizer, val_tweets)
val_labels= names_to_ids(val_labels)
h = model.fit(padded_train_seq, train_labels,validation_data=(val_seq, val_labels),epochs=20)


Epoch 1/20
500/500 [==============================] - 25s 50ms/step - loss: 0.0136 - accuracy: 0.9946 - val_loss: 0.5585 - val_accuracy: 0.8975
Epoch 2/20
500/500 [==============================] - 25s 50ms/step - loss: 0.0149 - accuracy: 0.9930 - val_loss: 0.5506 - val_accuracy: 0.8965
Epoch 3/20
500/500 [==============================] - 24s 49ms/step - loss: 0.0104 - accuracy: 0.9952 - val_loss: 0.5929 - val_accuracy: 0.8960
Epoch 4/20
500/500 [==============================] - 25s 49ms/step - loss: 0.0153 - accuracy: 0.9941 - val_loss: 0.5477 - val_accuracy: 0.8880
Epoch 5/20
500/500 [==============================] - 25s 49ms/step - loss: 0.0137 - accuracy: 0.9946 - val_loss: 0.5957 - val_accuracy: 0.8925
Epoch 6/20
500/500 [==============================] - 25s 50ms/step - loss: 0.0165 - accuracy: 0.9937 - val_loss: 0.7064 - val_accuracy: 0.8650
Epoch 7/20
500/500 [==============================] - 25s 50ms/step - loss: 0.0128 - accuracy: 0.9946 - val_loss: 0.5580 - val_accuracy:

In [31]:
#Evaluating and Testing Model

test_tweets, test_labels=get_tweet(test)
test_seq = get_sequences(tokenizer, test_tweets)
test_labels=names_to_ids(test_labels)
model.evaluate(test_seq, test_labels)

63/63 [==============================] - 1s 14ms/step - loss: 0.6404 - accuracy: 0.8945


[0.6403625011444092, 0.8945000171661377]

In [33]:
i = random.randint(0,len(test_labels)-1)
print('Sentence:', test_tweets[i])
print('Emotion:', index_to_class[test_labels[i]])
p = model.predict(np.expand_dims(test_seq[i], axis=0))[0]
print(test_seq[i])
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Predicted Emotion: ', pred_class)
#This code simply generates a random tweet and has the model predict what class it belongs to.
# It also predicts the tweet and its label so you can compare the prediction and the correct answer.


Sentence: i feel thats just strange on wotcs behalf
Emotion: surprise
[   2    3  311   32  234   30    1 3975    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
Predicted Emotion:  fear
